# Ranking heuristics & graph-based signals

Implement at least one non-learned ranking heuristic (such as popularity, recency, or graph-based propagation) to serve as a baseline for real systems. Analyze its bias

We implemented three distinct rankers:
1. `PopularityRanker`: global frequency-based baseline
2. `RecencyRanker`: time-decayed popularity model
3. `PersonalizedPageRankRanker` :  graph-based propagation method running on the user-item bipartite network
These were evaluated using our custom `RecommenderEvaluator` framework from HW1

We reuse the `MovieLensDataLoader` and train/val/test splits established in our previous evaluation framework, making them directly comparable to our MF models

In [1]:
import sys
sys.path.append("../src")
sys.path.append("../src/evaluation")
sys.path.append("../src/models")

import numpy as np
import pandas as pd
from scipy.sparse import csr_matrix, csc_matrix
from collections import defaultdict
from utils.data_loader import MovieLensDataLoader
from evaluator import RecommenderEvaluator

loader = MovieLensDataLoader()
train, val, test = loader.load_splits()
_, movies, _ = loader.load_raw_data()

Train: 797,758 | Val: 97,383 | Test: 105,068
Loaded 1,000,209 ratings
Loaded 3,883 movies
Loaded 6,040 users


**Popularity-Based Ranking**

It ranks items solely by their global interaction count in the training set. It filters out items with fewer than a minimum threshold of ratings to avoid noise


In [2]:
class PopularityRanker:
    
    def __init__(self, train_df, min_ratings=5):
        # count total interactions for each item
        counts = train_df.groupby('item_id').size()

        # store counts as a dictionary for fast lookups
        self.scores = counts.to_dict()
        self.min_ratings = min_ratings

        # filter out unpopular items
        self.ranked = counts[counts >= min_ratings].sort_values(ascending=False).index.tolist()
    
    def predict_for_user(self, user_id, k=10, train_df=None):
        seen = set()

        # extract items the user has already interacted with
        if train_df is not None:
            seen = set(train_df[train_df['user_id'] == user_id]['item_id'])
        # keep items the user has not seen
        recs = [(iid, self.scores[iid]) for iid in self.ranked if iid not in seen]

        # return top-K
        return recs[:k]

**Recency-Weighted Ranking**

It applies an exponential decay to historical interactions based on the timestamp. We set a half-life of 90 days. We check whether there is strong temporal drift

In [3]:
class RecencyRanker:
    
    def __init__(self, train_df, half_life_days=90, min_ratings=5):
        df = train_df.copy()

        # find the most recent timestamp to act as the current time
        max_ts = df['timestamp'].max()

        # calculate how many days ago each interaction occurred
        days_ago = (max_ts - df['timestamp']) / 86400

        # apply exponential decay (older ratings get exponentially lower weights)
        df['weight'] = np.exp(-np.log(2) * days_ago / half_life_days)
        
        # sum weights for each item to get its final score
        item_scores = df.groupby('item_id')['weight'].sum()

        # filtering with low num of interactions
        item_counts = df.groupby('item_id').size()
        item_scores = item_scores[item_counts >= min_ratings]
        
        self.scores = item_scores.to_dict()
        self.ranked = item_scores.sort_values(ascending=False).index.tolist()
    
    def predict_for_user(self, user_id, k=10, train_df=None):
        seen = set()
        # extract items the user has already interacted with
        if train_df is not None:
            seen = set(train_df[train_df['user_id'] == user_id]['item_id'])
        # keep items the user has not seen
        recs = [(iid, self.scores[iid]) for iid in self.ranked if iid not in seen]

        # return top-K
        return recs[:k]

**Graph-Based Propagation (personalized pagerank)**

It constructs a bipartite graph of users and items connected by positive interactions (rating >= 4.0). It simulates a random walk starting from the target user, propagating preference scores through the network with a teleportation probability (`alpha=0.15`)

In [4]:
class PersonalizedPageRankRanker:

    def __init__(self, train_df, alpha=0.15, n_iterations=20, threshold=4.0):
        self.alpha = alpha
        self.n_iter = n_iterations
        self.threshold = threshold
        
        # filteting only positive signals (>=4.0)
        df = train_df[train_df['rating'] >= threshold]

        # dimensions for the bipartite graph (users + items)
        self.n_users = train_df['user_id'].max() + 1
        self.n_items = train_df['item_id'].max() + 1
        n = self.n_users + self.n_items
        
        # create undirected edges: user -> item AND item -> user
        rows = np.concatenate([df['user_id'].values, self.n_users + df['item_id'].values])
        cols = np.concatenate([self.n_users + df['item_id'].values, df['user_id'].values])
        data = np.ones(len(rows), dtype=np.float32)
        # build sparse adjacency matrix
        adj = csr_matrix((data, (rows, cols)), shape=(n, n))
        
        # calculate out-degree for each node to normalize transition probabilities
        degree = np.array(adj.sum(axis=1)).flatten()
        degree[degree == 0] = 1
        inv_deg = 1.0 / degree
        # row-normalized stochastic transition matrix
        self.transition = csr_matrix((inv_deg[adj.nonzero()[0]] * adj.data, adj.indices, adj.indptr), shape=(n, n))
        
        # items user has interacted with
        self.user_seen = {}
        for uid, g in train_df.groupby('user_id'):
            self.user_seen[uid] = set(g['item_id'])
    
    def run_ppr(self, user_id):
        n = self.n_users + self.n_items
        # vector concentrated on target user
        teleport = np.zeros(n, dtype=np.float32)
        teleport[user_id] = 1.0
        
        scores = teleport.copy()
        # power iteration
        for _ in range(self.n_iter):
            scores = (1 - self.alpha) * (self.transition.T @ scores) + self.alpha * teleport
        # return only the scores corresponding to item nodes
        return scores[self.n_users:]
    
    def predict_for_user(self, user_id, k=10, train_df=None):
        if user_id >= self.n_users:
            return []
        # get raw PPR scores for all items
        item_scores = self.run_ppr(user_id)
        seen = self.user_seen.get(user_id, set())
        # masking out seen items
        for iid in seen:
            if iid < len(item_scores):
                item_scores[iid] = -np.inf
        
        # partition and top-K
        top_k = np.argpartition(item_scores, -k)[-k:]
        top_k = top_k[np.argsort(item_scores[top_k])[::-1]]
        return [(int(i), float(item_scores[i])) for i in top_k if item_scores[i] > -np.inf]

**Model Evaluation**

We evaluate all three rankers using our `RecommenderEvaluator`. We look specifically at NDCG, Recall, and Coverage/Popularity bias to understand the distribution of the recommendations

In [5]:
pop = PopularityRanker(train)
rec = RecencyRanker(train, half_life_days=90)
ppr = PersonalizedPageRankRanker(train, alpha=0.15, n_iterations=20)

In [6]:
evaluator = RecommenderEvaluator(train, test, k_values=[5, 10, 20])

for name, model in [("Popularity", pop), ("Recency", rec), ("PPR", ppr)]:
    print(f"\n")
    metrics = evaluator.evaluate_model(model, model_name=name)
    evaluator.print_metrics(metrics, model_name=name)

evaluator.save_results("../experiments/results/heuristic_results.csv")



Popularity - Evaluation results
Ranking metrics:
NDCG@ 5: 0.0437
NDCG@10: 0.0490
NDCG@20: 0.0615


Relevance metrics (threshold=4.0):
Recall@ 5: 0.0243
Precision@ 5: 0.0425
Recall@10: 0.0433
Precision@10: 0.0393
Recall@20: 0.0802
Precision@20: 0.0352


Diversity metrics:
Coverage: 0.0513
Popularity bias: 2085.91


Recency - Evaluation results
Ranking metrics:
NDCG@ 5: 0.0380
NDCG@10: 0.0438
NDCG@20: 0.0547


Relevance metrics (threshold=4.0):
Recall@ 5: 0.0215
Precision@ 5: 0.0383
Recall@10: 0.0410
Precision@10: 0.0353
Recall@20: 0.0744
Precision@20: 0.0315


Diversity metrics:
Coverage: 0.0436
Popularity bias: 1796.94


PPR - Evaluation results
Ranking metrics:
NDCG@ 5: 0.0475
NDCG@10: 0.0544
NDCG@20: 0.0667


Relevance metrics (threshold=4.0):
Recall@ 5: 0.0262
Precision@ 5: 0.0463
Recall@10: 0.0497
Precision@10: 0.0428
Recall@20: 0.0875
Precision@20: 0.0375


Diversity metrics:
Coverage: 0.0578
Popularity bias: 2052.98


In [7]:
results_df = pd.DataFrame(evaluator.history)
cols = ['Model', 'NDCG@5', 'NDCG@10', 'NDCG@20', 'Recall@10', 'Coverage', 'Popularity_Bias']
print(results_df[[c for c in cols if c in results_df.columns]].to_string(index=False))

     Model   NDCG@5  NDCG@10  NDCG@20  Recall@10  Coverage  Popularity_Bias
Popularity 0.043673 0.049011 0.061451   0.043306  0.051282      2085.907599
   Recency 0.037991 0.043834 0.054690   0.040981  0.043644      1796.937525
       PPR 0.047532 0.054392 0.066679   0.049691  0.057829      2052.977525


**Analysis**

##### Popularity-Based Ranking
* assumes a "wisdom of the crowd" - if many people like it, it has universal quality
* effective for pure cold-start scenarios or mainstream users. It proved to be a strong baseline on our dataset (NDCG@10 = 0.049)
* fails for niche users. It also has poor catalog discovery (only 5.1% coverage)

##### Recency-Weighted Popularity
* encodes a temporal bias - assumes user preferences drift quickly and recent interactions are more relevant
* fails on static catalogs. In MovieLens, penalizing classic movies simply because they are old actually degraded performance compared to raw popularity (NDCG@10 = 0.0438)

##### Personalized PageRank
* encodes a transitive similarity bias via graph topology - assumes preference flows through user-item community clusters
* very competitive when the interaction graph is dense. PPR was our best-performing heuristic (NDCG@10 = 0.0544). It captured collaborative filtering signals and achieved the highest catalog coverage (5.78%)
* fails on isolated cold-start items where the random walker cannot reach. It is also computationally expensive to run at inference time compared to simple popularity lookups